In [1]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
from scipy.sparse import csr_matrix, lil_matrix, load_npz
import os

In [2]:
tuner_output_images = '../datapreperation/images_files/output_files/image_model'


In [3]:
path = os.path.join(tuner_output_images, 'matrices/cls/cls_T11_fold_vector.npy')
folds = np.load(path)

In [4]:
### Uncomment if y matrix is stored in npy format
#path = os.path.join(tuner_output_images, 'matrices/cls/cls_T10_y.npy')
#t10 = np.load(path,allow_pickle=True).item()

path = os.path.join(tuner_output_images, 'matrices/cls/cls_T10_y.npz')
t10 = load_npz(path).tocsr()

In [5]:
out = lil_matrix(t10) 
out

<9788x284 sparse matrix of type '<class 'numpy.float32'>'
	with 209384 stored elements in List of Lists format>

In [6]:
# fold 2

In [7]:
fold = 2 
out[folds != fold, :] = 0 

In [8]:
# remove auxiliary tasks (aggregation weight 0)

In [33]:
path = os.path.join(tuner_output_images, 'results_tmp/classification/T8c.csv')
t8c = pd.read_csv(path)

In [10]:
cctis = np.array(t8c.query('is_auxiliary == False').query('aggregation_weight > 0')['cont_classification_task_id'],dtype=int)

In [19]:
cctis.shape

(112,)

In [12]:
os.makedirs('files', exist_ok=True)

In [13]:
np.save('./files/main_tasks_with_image.npy',cctis)

In [14]:
# mind the ~
mask = ~np.isin(
    np.array(range(int(t8c['cont_classification_task_id'].max()+1)))
    ,cctis
)

In [15]:
out[:,mask].shape

(9788, 172)

In [16]:
# ignore all non-main tasks 
out[:,mask] = 0 

In [17]:
filename = './files/y_sparse_step1_main_tasks_fold2.npy'
np.save(filename,csr_matrix(out))